# Query result content

Queries return a `Result` object that we can customize according to our needs.

In [1]:
%pip install superlinked==18.8.0

In [2]:
import pandas as pd
from superlinked import framework as sl

pd.set_option("display.max_colwidth", 100)

Let's create a simple config and ingest two entities of text just for demonstration purposes.

In [3]:
class Paragraph(sl.Schema):
    id: sl.IdField
    body: sl.String
    like_count: sl.Integer


paragraph = Paragraph()

body_space = sl.TextSimilaritySpace(text=paragraph.body, model="sentence-transformers/all-mpnet-base-v2")
like_space = sl.NumberSpace(number=paragraph.like_count, min_value=0, max_value=100, mode=sl.Mode.MAXIMUM)
# indices can be built on top of multiple spaces as simple as that
paragraph_index = sl.Index([body_space, like_space])

source: sl.InMemorySource = sl.InMemorySource(paragraph)
executor = sl.InMemoryExecutor(sources=[source], indices=[paragraph_index])
app = executor.run()

source.put(
    [
        {
            "id": "paragraph-1",
            "body": "Glorious animals live in the wilderness.",
            "like_count": 75,
        },
        {
            "id": "paragraph-2",
            "body": "Growing computation power enables advancements in AI.",
            "like_count": 10,
        },
    ]
)

### .select_all()

This clause makes the query result return all the fields the entities have.

In [4]:
query_base = (
    sl.Query(
        paragraph_index,
        weights={
            body_space: 1.0,
            like_space: 1.0,
        },
    )
    .find(paragraph)
    .similar(body_space, "What makes the AI industry go forward?")
)

query_select_all = query_base.select_all()

A query without any select clause will return none of the fields.

In [5]:
result_no_select_clause = app.query(query_base)
result_no_select_clause.entries[0].entity.fields

[]

Notice all 2 fields (both `body` and `like_count` of `paragraph`) are returned with the `.select_all()` clause.

In [6]:
result_select_all = app.query(query_select_all)
result_select_all.entries[0].entity.fields

[ParsedSchemaField of (String(name=body, type=<class 'str'>, schema_object_name=Paragraph)),
 ParsedSchemaField of (Integer(name=like_count, type=<class 'int'>, schema_object_name=Paragraph))]

### .select()

The select clause enables fine-grained field selection.

In [7]:
query_fix_select = query_base.select(paragraph.body)
result_fix_select = app.query(query_fix_select)
result_fix_select.entries[0].entity.fields

[ParsedSchemaField of (String(name=body, type=<class 'str'>, schema_object_name=Paragraph))]

It can also be used with an `sl.Param`, filled query time.

In [8]:
query_select = query_base.select(sl.Param("select_fields"))

Fields can be referenced via string names,

In [9]:
result_body = app.query(query_select, select_fields=["body"])
result_body.entries[0].entity.fields

[ParsedSchemaField of (String(name=body, type=<class 'str'>, schema_object_name=Paragraph))]

or referring to the schema field itself.

In [10]:
result_like = app.query(query_select, select_fields=[paragraph.like_count])
result_like.entries[0].entity.fields

[ParsedSchemaField of (Integer(name=like_count, type=<class 'int'>, schema_object_name=Paragraph))]

An empty select clause will result in no fields are returned for efficiency (ids are always returned of course),

In [11]:
result_empty = app.query(query_select)
result_empty.entries[0].entity.fields

[]

and referencing ways can even be mixed in the same clause.

In [12]:
result_both = app.query(query_select, select_fields=[paragraph.body, "like_count"])
result_both.entries[0].entity.fields

[ParsedSchemaField of (String(name=body, type=<class 'str'>, schema_object_name=Paragraph)),
 ParsedSchemaField of (Integer(name=like_count, type=<class 'int'>, schema_object_name=Paragraph))]